In [9]:
# Install necessary libraries
# %conda install pytorch::pytorch torchvision torchaudio -c pytorch
%conda install pandas
%pip install faster-whisper
%pip install pydub
# %conda install transformers datasets evaluate accelerate
# %conda install pandas matplotlib seaborn ipywidgets
# %conda install scikit-learn -c conda-forge scikit-learn
# %pip install bertviz

Channels:
 - defaults
 - conda-forge
 - pytorch
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
# What version of Python do you have?
import sys
import platform
import torch

has_gpu = torch.cuda.is_available()
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print("NVIDIA/CUDA GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

torch.cuda.empty_cache()

Python Platform: macOS-14.5-arm64-arm-64bit
PyTorch Version: 2.4.0

Python 3.11.9 (main, Apr 19 2024, 11:43:47) [Clang 14.0.6 ]
NVIDIA/CUDA GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [11]:
import pathlib
from pathlib import Path

# path to mp3 audio files
path_to_mp3_audio_folder = "mp3_audio_files/"

# path to transcripts
path_to_transcripts_folder = path_to_mp3_audio_folder + "transcripts/"

if not Path(path_to_transcripts_folder).is_dir():
    Path(path_to_transcripts_folder).mkdir(parents=True, exist_ok=True)

In [12]:
p = Path(path_to_mp3_audio_folder).glob("**/*")
mp3_audio_files = [x for x in p if x.is_file() and ".mp3" in x.name]
len(mp3_audio_files), mp3_audio_files[0]

(1, PosixPath('mp3_audio_files/lesson1_of_RAG_course_with_DeepLearningAI.mp3'))

In [13]:
import torch
from faster_whisper import WhisperModel

# model_size = "large-v2"
model_size = "large-v3"

# get device
device = "cuda:0" if torch.cuda.is_available() else "cpu"

if device == "cuda:0":
    # Run on GPU with FP16
    model = WhisperModel(model_size, device="cuda", compute_type="float16")
    # or Run on GPU with INT8
    # model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
else:
    # Run on CPU with INT8
    model = WhisperModel(model_size, device="cpu", compute_type="int8")

/Users/gsampaio/redhat/ai/llm-agents/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# check GPU (cuda) or CPU
device

'cpu'

In [17]:
import pandas as pd

# %%time

for i, path_to_audio_file in enumerate(mp3_audio_files[:1]):

    # get all audio segments
    # from pydub.playback import play
    segments, _ = model.transcribe(str(path_to_audio_file), beam_size=5, vad_filter=True)
    # segments = list(segments)  # The transcription will actually run here.

    # get audio language
    # print("Detected language '%s' with probability %f with faster whisper\n" % (info.language, info.language_probability))

    # save audio segments with start and end time, and transcript by audio segment
    start_segments, end_segments, text_segments = list(), list(), list()
    for segment in segments:
        # print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
        start, end, text = segment.start, segment.end, segment.text
        start_segments.append(start)
        end_segments.append(end)
        text_segments.append(text)

    # save transcript into csv
    df = pd.DataFrame()
    df["start"] = start_segments
    df["end"] = end_segments
    df["text"] = text_segments
    path_to_audio_file_transcript = path_to_transcripts_folder + path_to_audio_file.name.replace(".mp3", ".csv").replace(".wav", ".csv")
    df.to_csv(path_to_audio_file_transcript, encoding='utf-8', index=False)

    if i % 2 == 0: print(i)

0


In [19]:
df

,start,end,text
0,1.78,9.78,"Retrieval Augmented Generation, or RAG, has b..."
1,9.78,17.78,But to actually build and productionize a hig...
2,17.78,23.78,to give the LM highly relevant context to gen...
3,23.78,31.78,to help you efficiently iterate and improve y...
4,31.78,38.78,This course covers two advanced retrieval met...
...,...,...,...
65,267.78,268.78,Sounds great.
66,268.78,269.78,Let's get started.
67,269.78,274.78,And I think you'll be able to really clean up...
68,274.78,276.78,Laughed on it.


In [18]:
import nltk

# Download the Punkt tokenizer
nltk.download("punkt")

ModuleNotFoundError: No module named 'nltk'

In [ ]:
paragraph = " ".join(df["text"].tolist()).replace("  ", " ")

# Tokenize the paragraph into sentences
sentences = nltk.sent_tokenize(paragraph)

In [ ]:
css = """
        
      """

from IPython.display import display, HTML

for sentence in sentences:
    display(HTML(f"{css} {sentence}"))